# Interpolate and transform (rotate and translate) navigation data

In [ ]:
# import the navigation module
import themachinethatgoesping.navigation as nav

## Setup sensor configuration object
This object stores the offsets of all used navigation sensors and of the target sensor (e.g. the MBES).

In [ ]:
# initialize
scs = nav.SensorConfiguration()

# add sensor offsets
scs.set_heading_source("compass",9)                      # yaw
scs.set_depth_source("altitude-meter",0, 0, 1)           # x, y, z
scs.set_position_source("gps",1, 2, 3)        # x, y, z
scs.set_attitude_source("IMU",10, -10, -30)   # yaw, pitch, roll

# add a target sensor with offsets
scs.add_target(
  target_id = "mbes",
  x = 1,     # in m, positive forward
  y = 2,     # in m, positive starboard
  z = 3,     # in m, positive down
  yaw = 0,   # in degrees, positive clockwise
  pitch = 0, # in degrees, positive bow up
  roll = 0)  # in degrees, positive port up

# Note(1): The offsets are relative to the origin of the vessels coordinate system.

## Add navigation data
First a NavigationInterpolator object must be initialzied.

The user has two options: 
- NavigationInterpolatorLocal stores norhting and easting (in meter)
- NavigationInterpolatorLatLon stores and interpolates latitudes and longitudes (in °)

In this example we use latitude and longitude.

In [ ]:
#example data
# pos sys
timestamps_possys = [10,   20  , 30]   # time since epoch (in s)
latitude          = [54.0, 54.1, 54.2] # (in °)
longitude         = [10.0, 10.1, 10.2] # (in °)

# attitude sensor
timestamps_attitude = [  1, 15, 22, 32] # time since epoch (in s)
yaw                 = [-1 ,  1, 10,  5] # clockwise rotation (in °) (0° is north)
pitch               = [0.1, -2,  1,  2] # positve is bow up (in °)
roll                = [1  ,  2,  3, -4] # positive is port up (in °)

In [ ]:
# initialize the NavigationInterpolator using the previously created sensor configuration
navi = nav.NavigationInterpolatorLatLon(scs)

# add data
navi.set_data_position(timestamps_possys, latitude, longitude)
navi.set_data_attitude(timestamps_attitude, pitch, roll)
navi.set_data_heading(timestamps_attitude, yaw)

# Note: data that are not set are assumed to be 0.0 (e.g depth in the above example)

## Get sensor position and attitude for random ping time points

The user can now retrieve the position and attitude of the registered target sensors (add_target) as Geolocation object. 

The returned type of Geolocation (GeolocationLocal or GeolocationLatLon) is determined by the type of the navigation interpolator (NavigationInterpolatorLocal or NavigationInterpolatorLatLon)

In [ ]:
# get the location of the "mbes" at timestamp 25 (seconds since 1970)
location_mbes = navi.compute_target_position("mbes",timestamp=25)

print(location_mbes)

Note: the compute_target_position time stamps can exceed the time stamps of data inside the NavigationInterpolator. 
The behavior is then determined by the extrapolation mode parameter (navi.set_extrapolation_mode()).
This parameter can be:

1. "extrapolate" (default): The data is extrapolated (linear continuation of the used akima spline)
2. "nearest": The last (or respective first) data point is used
3. "fail": An exception is thrown if the time stamp is outside the data range

## UTM/LatLon conversion of Geolocation objects
The NavigationInterpolatorLatLon object computes GeolocationLatLon objects.
This stores latitude and longitude values in °.

The NavigationInterpolatorLocal object computes GeolocationLocal objects.
This stores northing and easting values in m (without zone or hemisphere information)

In [ ]:
#location_mbes is a GeolocationLatLon object
print(location_mbes)

### ---

The Geolocation objects are part of the 'datastructures' name space and allow for implicit conversion.

GeolocationLatLon can be converted to GeolocationUTM:

In [ ]:
#convert this object to UTM
location_mbes_utm = nav.datastructures.GeolocationUTM(location_mbes)
print(location_mbes_utm)

Other conversion e.g. to/from GeolocationLocal are also possible. See the Geolocation module Api for details